In [1]:
from random import seed, randint
import ast; import sys; from time import process_time
import numpy as np; import verbose_module
from pickle import load, dump
from copy import deepcopy

sys.path.append('../../.')
import pIRPgym

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

########################     Instance generator and Environment     #########################
### pIRP model's parameters
# Stochasticity
stochastic_params = ['d','q']; look_ahead = ['d','q']

# Historical data
historical_data = ['*']

# Other parameters
backorders = 'backorders'

# Random Instance
q_params = {'dist': 'c_uniform', 'r_f_params': [80,110]}         # Supply
p_params = {'dist': 'd_uniform', 'r_f_params': [40,61]}        # Purchase costs at suppliers'

d_params = {'dist': 'log-normal-trunc', 'r_f_params': [3,1], 'break':104.5}          # Demand

h_params = {'dist': 'd_uniform', 'r_f_params': [20,40]}        # Holding costs

### Environment 
# Creating environment object
inventory = True; routing = True; perishability = "ages"
env = pIRPgym.steroid_IRP(routing, inventory, perishability)

# Environment Check

In [2]:
T = 7; verbose = True; agg = False; hold_cost = True; save = False
rr = 0.1; theta = 0.95; gamma = 1

In [3]:
w = [0,0,0,1,0]; seed_ix = 0
objectives = ["costs","climate","water","land","fossil"]

rewards = dict(); actions = dict(); la_decisions = dict(); states = dict(); backo = dict(); perished = dict(); norm_matrix = dict()
env_config = {'M':5,'K':2,'T':T,'F':2,'Q':375,'S':3,'LA_horizon':4,'d_max':2000,'hist_window':60, 'theta':theta, 'hold_cost':hold_cost, 'rr':rr, 'gamma':gamma}

stoch_rd_seed = 20; det_rd_seed = 10 # Random seeds
obj_weights = dict(zip(objectives,w))

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
inst_gen.generate_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

# Resetting the environment
states[1] = env.reset(inst_gen, return_state=True)

done = False; day = 1
while not done:

    ''' Compromise Programming '''
    pIRPgym.Compromise_Programming.normalize_objectives_IRP(inst_gen, env, verbose = verbose)
    norm_matrix[day] = deepcopy(env.norm_matrix)

    ''' Inventory Routing Problem '''
    action, la_decisions[day] = pIRPgym.Inventory.Stochastic_RH_IRP(states[day], env, inst_gen, objs=obj_weights, verbose = verbose)

    if verbose:
        if day == 1: verbose_module.headers.print_simple_header(w)
        print(f"\n------- Day {day}")
        verbose_module.objectives_performance.show_normalization(inst_gen,env)
        verbose_module.objectives_performance.show_balanced_solution(inst_gen,env,la_decisions[day])
    
    states[day+1], rewards[day], done, actions[day], (backo[day], perished[day]) = env.step(action,inst_gen,aggregated = agg)
    
    day += 1

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-07
---------------------------------------------------------------------------------------
- OPTIMIZING costs 
---------------------------------------------------------------------------------------
Sample path 0
	Purchasing decisions:
		P1 demand:	42.65	19.31	32.23	21.61
			Supplier 1:	['0.0', '0.0', '0.0', '0.0']
			Supplier 2:	['0.0', '48.67', '0.0', '21.61']
			Supplier 3:	['45.52', '0.0', '0.0', '0.0']
			Supplier 4:	['0.0', '0.0', '0.0', '0.0']
			Supplier 5:	['0.0', '0.0', '0.0', '0.0']
		P2 demand:	87.46	5.7	13.49	52.76
			Supplier 2:	['63.55', '19.19', '0.0', '52.76']
			Supplier 4:	['0.0', '0.0', '0.0', '0.0']
			Supplier 5:	['0.0', '0.0', '0.0', '0.0']
	Inventory Management:
		Product 1
			Fulfil.	[42.65, 0.0, 0.0]	[19.31, 0.0, 0.0]	[0.0, 29.36, 2.87]	[21.61, 0.0, 0.0]
			Invent.	[2.87, 0.0, 0.0]	[29.36, 2.87, 0.0]	[0.0, 0.0, 0.0]	[0.0, 0.0, 0.0]
			Perished		0.0		0.0		0.0		0.0
			Backord